애플 주식 데이터를 가지고 간단한 데이터 분석을 해보자. 모든 답은 Pyspark을 통해 이뤄져야 한다.

먼저 PySpark과 Py4J를 설치하자

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 43.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=b8ccad5e081c40d1a78d4e748770dd716bbad5333cee2763060427a21ab3962a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


#### Spark Session 만들기

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

#### 애플 주식 CSV 파일 로딩하기: https://pyspark-test-sj.s3-us-west-2.amazonaws.com/appl_stock.csv
일단 pandas 데이터프레임으로 로딩해서 Spark 데이터프레임으로 변경한다

In [4]:
import pandas as pd

apple_pandas_df = pd.read_csv("https://pyspark-test-sj.s3-us-west-2.amazonaws.com/appl_stock.csv")
apple_spark_df = spark.createDataFrame(apple_pandas_df)

#### 1> 어떤 컬럼 이름들이 있는가?

In [5]:
apple_spark_df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### 2> 스키마를 프린트해보기

In [6]:
apple_spark_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Adj Close: double (nullable = true)



#### 3> 처음 5개의 레코드를 출력해보기

In [7]:
apple_spark_df.show(n=5)

+----------+----------+----------+----------+----------+---------+------------------+
|      Date|      Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+----------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.380001|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|213.249994|214.379993|150476200|         27.774976|
|2010-01-06|214.379993|    215.23|210.750004|210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|209.050005|    210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.060005|211.980005|111902700|         27.464034|
+----------+----------+----------+----------+----------+---------+------------------+
only showing top 5 rows



#### 4> describe를 사용하여 데이터프레임의 컬럼별 통계보기

In [8]:
apple_spark_df.describe().show()

+-------+----------+------------------+-----------------+------------------+------------------+--------------------+------------------+
|summary|      Date|              Open|             High|               Low|             Close|              Volume|         Adj Close|
+-------+----------+------------------+-----------------+------------------+------------------+--------------------+------------------+
|  count|      1762|              1762|             1762|              1762|              1762|                1762|              1762|
|   mean|      null|313.07631115890996|315.9112880164584|309.82824050794557| 312.9270656379114| 9.422577587968218E7| 75.00174115607273|
| stddev|      null|185.29946803981545|186.8981768648577|183.38391664370988|185.14710361709427|6.0205187765927084E7|28.574929721799037|
|    min|2010-01-04|              90.0|        90.699997|         89.470001|         90.279999|            11475900|         24.881912|
|    max|2016-12-30|        702.409988|       70

#### 5> Close 컬럼의 평균값은 얼마인가?

In [9]:
from pyspark.sql.functions import mean

apple_spark_df.select(mean("Close")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|312.9270656379114|
+-----------------+



#### 6> Volume 컬럼의 최대값과 최소값은?

In [10]:
from pyspark.sql.functions import min, max

apple_spark_df.select(max("Volume"), min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|  470249500|   11475900|
+-----------+-----------+



#### 보너스 질문: HV ratio라는 이름의 새로운 컬럼을 추가한 데이터프레임을 만들기. 이 컬럼의 값은 High/Volume으로 계산된다

In [ ]:
apple_spark_df_with_hv = apple_spark_df.withColumn("hv ratio", apple_spark_df.High/apple_spark_df.Volume) 

In [ ]:
apple_spark_df_with_hv.show(5)

#### 보너스 질문: 월별 Close 컬럼의 평균값은?

In [ ]:
from pyspark.sql.functions import month

monthdf = apple_spark_df.withColumn("Month", month("Date"))

In [ ]:
monthavgdf = monthdf.select(["Month", "Close"]).groupBy("Month").mean()

In [ ]:
monthavgdf.show()

In [ ]:
monthavgdf.select(["Month", "avg(Close)"]).orderBy("Month").show()